In [1]:
import numpy as np
from torchvision.datasets import MNIST

In [2]:
train = MNIST(root='./data', train = True, download = True, transform = None)
test = MNIST(root='./data', train = False, download = True, transform = None)

100%|██████████| 9912422/9912422 [00:00<00:00, 87931557.84it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 28027694.08it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 23596798.88it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3249280.02it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
# Data format = ( PIL image, annotation )

print(f"TRAIN - {type(train)} {len(train)}")
print(f"TEST - {type(test)} {len(test)}")

TRAIN - <class 'torchvision.datasets.mnist.MNIST'> 60000
TEST - <class 'torchvision.datasets.mnist.MNIST'> 10000


In [4]:
def to_categorical(input, num_classes):
  output = []
  for image, label in input:
    labelc = [0] * num_classes
    labelc[label] = 1
    output.append( (image, labelc) )
  return output

def from_categorical(input):
  output = []
  for x in input:
    output.append([np.argmax(x)])
  return output

def preprocess_and_flatten(input):
  output = []
  for image, label in input:
    arr = (np.array(image).flatten()/255).reshape(784,1)
    output.append((arr, label))
  return output

In [6]:
train_final = to_categorical(train, 10)
train_final[0][0].show()
train_flat = preprocess_and_flatten(train_final)

train_split = train_flat[:50000]
test_split = train_flat[50000:]

print(len(train_split))
print(len(test_split))
# print(train_final[0][1], train[0][1])

50000
10000


In [7]:
class Layer:
  def __init__(self):
    self.input_size = None
    self.output_size = None

  def forward(self, input):
    print("Not Implemented in derived Layer")
    pass

  def backward(self, input):
    print("Not Implemented in derived Layer")
    pass

In [9]:
class Activation(Layer):
  def __init__(self, activation, activation_derivative):
    self.X = []

    self.activation = activation
    self.activation_derivative = activation_derivative

  def forward(self, input):
    self.X = input
    return self.activation(input)

  def backward(self, del_E_by_del_Y, learning_rate):
    return np.multiply(del_E_by_del_Y, self.activation_derivative(self.X))

# activation = Activation(lambda x: np.tanh(x), lambda x: 1- np.tanh(x)**2)
# sample_input = np.random.rand(4,1)

# print("Activation forward", activation.forward(sample_input))

# sample_grad = np.random.rand(4,1)

# print("Activation backprop", activation.backward(sample_grad, 0.01))

In [10]:
class Tanh:
  def loss(self, x):
    return np.tanh(x)

  def loss_derivative(self, x):
    return 1-np.tanh(x)**2

  def __init__(self):
    self.X = []

    self.activation = self.loss
    self.activation_derivative = self.loss_derivative

  def forward(self, input):
    self.X = input
    return self.activation(input)

  def backward(self, del_E_by_del_Y, learning_rate):
    return np.multiply(del_E_by_del_Y, self.activation_derivative(self.X))

# test = reLu()

# print(reLu.loss_derivative(5.1))

In [12]:
class Dense(Layer):
  def __init__(self, input_length :int, output_length :int):

    self.X = []

    self.W = np.random.rand(output_length, input_length) - 0.5
    self.B = np.random.rand(output_length, 1) - 0.5

    # print(f"initialized weights {self.W}")
    # print(f"initialized bias {self.B}")

  def forward(self, X_input):
    self.X = X_input
    # print(f"Dense Layer forward {self.X.shape}")
    # print(f"Weights shape {self.W.shape} Bias shape {self.B.shape} Mult shape {np.dot(self.W,X_input).shape}")
    # print(f"Weights shape {self.W.shape} Bias shape {self.B.shape} Mult shape {np.dot(self.W,X_input).shape}")
    # print(f"Weights {self.W} Bias {self.B} Mult {np.dot(self.W,X_input)}")
    # print(f"Forward output {(np.add((np.dot(self.W,X_input)),self.B)).shape}")
    return self.W @ X_input + self.B

  def backward(self, del_E_by_del_Y, learning_rate):
    # print(f"Dense backward grad {del_E_by_del_Y.shape} self shape {self.X.shape}")
    del_E_by_del_W = np.dot(del_E_by_del_Y, self.X.T)
    del_E_by_del_B = del_E_by_del_Y

    self.learning_rate = learning_rate

    self.W = self.W - self.learning_rate * del_E_by_del_W
    self.B = self.B - self.learning_rate * del_E_by_del_B

    del_E_by_del_X = np.dot(self.W.T,del_E_by_del_Y)

    return del_E_by_del_X


# layer = Dense(3, 5)
# sample_input = np.random.rand(3,1)
# print(sample_input)
# print("Forward prop", layer.forward(sample_input))
# sample_gradient = np.random.rand(5,1)
# print(sample_gradient)
# print("backward prop", layer.backward(sample_gradient, 0.01))

In [13]:
class Model:
  def __init__(self):
    self.layers = []
    self.loss = None
    self.loss_derivative = None

  def add(self, layer):
    self.layers.append(layer)

  def setloss(self, loss, loss_derivative):
    self.loss = loss
    self.loss_derivative = loss_derivative

  def predict(self, input_list):
    num_samples = len(input_list)
    results = []

    outputs = []
    for input in input_list:
      out = input
      for layer in self.layers:
        out = layer.forward(out)
      outputs.append(out)

    return outputs

  def train(self, input_tuple, epochs, learning_rate):
    for epoch in range(epochs):
      error = 0

      accuracy = 0
      for input, value in input_tuple:
        out = input
        # print(f"Initial {out.shape}")
        for layer in self.layers:
          out = layer.forward(out)
          # print(f"New out {out.shape}")

        loss_val = self.loss(out, value)

        error += loss_val


        accuracy += np.array_equal(np.argmax(value), np.argmax(out))
        # print(f"Output {np.argmax(value)} {np.argmax(out)} {np.array_equal(np.argmax(value), np.argmax(out))}")
        # print(f"Output array \n{value} \n{out} \n{self.loss(out,value)}")
        # print(f"Output shape {out.shape}")
        grad = -self.loss_derivative(out, value)

        # print(f"Grad shape {grad.shape}")
        for layer in reversed(self.layers):
          # print(type(layer))
          grad = layer.backward(grad, learning_rate)

      error/=len(input_tuple)

      accuracy/=len(input_tuple)

      print(f"Epoch {epoch+1} Error {error} Accuracy {accuracy}")

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    true_len = len(y_true)
    y_true = np.array(y_true).reshape(true_len, 1)

    pred_len = len(y_pred)
    y_pred = np.array(y_pred).reshape(pred_len, 1)
    # print(f"mse_prime {y_true.shape, y_pred.shape}")
    return 2*(y_pred-y_true)/y_true.size;

# model = Model()
# model.add(Dense(784, 20))
# model.add(Tanh())
# model.add(Dense(20, 20))
# model.add(Tanh())
# model.add(Dense(20, 10))
# model.add(Tanh())
# model.setloss(mse, mse_prime)

# model.predict([np.random.rand(784,1)])

In [14]:
# Check if the model is learning XOR

# print(train_flat[:1])
model = Model()
model.add(Dense(2, 3))
model.add(Tanh())
model.add(Dense(3, 2))
model.add(Tanh())
# model.add(Tanh())
model.setloss(mse, mse_prime)

model.train([[np.array([[0],[0]]), np.array([[1],[0]])], [np.array([[0],[1]]), np.array([[0],[1]])], [np.array([[1],[0]]), np.array([[0],[1]])], [np.array([[1],[1]]), np.array([[1],[0]])]], 5000, 0.01)

Epoch 1 Error 0.8512123882174714 Accuracy 0.75
Epoch 2 Error 0.7964912474080548 Accuracy 0.75
Epoch 3 Error 0.7448717952522498 Accuracy 0.75
Epoch 4 Error 0.6966178880443644 Accuracy 0.75
Epoch 5 Error 0.6518900008813981 Accuracy 0.75
Epoch 6 Error 0.6107507094277658 Accuracy 0.75
Epoch 7 Error 0.573175609338086 Accuracy 0.75
Epoch 8 Error 0.539067728123419 Accuracy 0.75
Epoch 9 Error 0.5082734944746807 Accuracy 0.75
Epoch 10 Error 0.4805986195037247 Accuracy 0.75
Epoch 11 Error 0.45582268123620434 Accuracy 0.75
Epoch 12 Error 0.43371166900819086 Accuracy 0.75
Epoch 13 Error 0.41402815566176276 Accuracy 0.75
Epoch 14 Error 0.39653908066928434 Accuracy 0.75
Epoch 15 Error 0.38102133734688926 Accuracy 0.75
Epoch 16 Error 0.3672654736802903 Accuracy 0.75
Epoch 17 Error 0.3550778601845284 Accuracy 0.75
Epoch 18 Error 0.3442816726068964 Accuracy 0.75
Epoch 19 Error 0.3347170023108975 Accuracy 0.75
Epoch 20 Error 0.3262403583070064 Accuracy 0.75
Epoch 21 Error 0.31872377270884644 Accuracy 0.

In [16]:
# Train the model on MNIST

model2 = Model()
model2.add(Dense(784,40))
model2.add(Tanh())
model2.add(Dense(40,20))
model2.add(Tanh())
model2.add(Dense(20,10))

model2.setloss(mse, mse_prime)

train_split = train_flat[:50000]
test_split = train_flat[50000:]

model2.train(train_split, 100, 0.01)

Epoch 1 Error 0.1162983961383484 Accuracy 0.403
Epoch 2 Error 0.13722751148269718 Accuracy 0.68346
Epoch 3 Error 0.1471059428221899 Accuracy 0.78164
Epoch 4 Error 0.15424559184752012 Accuracy 0.84774
Epoch 5 Error 0.15873306624563635 Accuracy 0.872
Epoch 6 Error 0.16086491083333804 Accuracy 0.8837
Epoch 7 Error 0.16225447274289387 Accuracy 0.89148
Epoch 8 Error 0.16329078291409616 Accuracy 0.89776
Epoch 9 Error 0.16411855302040818 Accuracy 0.90274
Epoch 10 Error 0.16480744800555516 Accuracy 0.9068
Epoch 11 Error 0.1653941307888644 Accuracy 0.91084
Epoch 12 Error 0.16589985196459642 Accuracy 0.914
Epoch 13 Error 0.16634013406780884 Accuracy 0.91716
Epoch 14 Error 0.16672868588332876 Accuracy 0.91976
Epoch 15 Error 0.16707629430370005 Accuracy 0.92182
Epoch 16 Error 0.16739051571395444 Accuracy 0.92394
Epoch 17 Error 0.16767672558143815 Accuracy 0.92588
Epoch 18 Error 0.1679392460458763 Accuracy 0.92784
Epoch 19 Error 0.168181639536482 Accuracy 0.92958
Epoch 20 Error 0.16840659718971987 

In [19]:
def evaluate(input, model):
  length = len(input)
  accuracy = 0
  for test_case, output in input:
    prediction = model.predict([test_case])
    # print(np.argmax(prediction), np.arg)
    accuracy+=np.array_equal(np.argmax(prediction), np.argmax(output))
  return accuracy/length

print(evaluate(test_split, model2))

0.95


In [24]:
test_final = to_categorical(test, 10)
test_flat = preprocess_and_flatten(test_final)

print(len(test_flat))

test_final[4][0].show()

10000


In [26]:
test_final[0][0].show()

print(from_categorical(model2.predict([test_flat[0][0]])))

[[7]]
